In [ ]:
%%bash
pip install -r requirements.txt

## INSERT YOUR META-ANALYSIS ID HERE

In [ ]:
META_ID = "3SD2A7LGuQTm"

In [ ]:
from nimare.workflows import compose_run
from nilearn.plotting import plot_stat_map

In [ ]:
if META_ID is not None:
    meta_result = compose_run(META_ID)
else:
    print("UPDATE META_ID VARIABLE TO YOUR META-ANALYSIS")

## Choose which statistical map to investigate
default is z map, but each estimator/corrector combination may generate corrected/more appropriate statistical maps for summary/investigation

In [ ]:
# here are your choices
print("\n".join(list(meta_result.maps.keys())))

In [ ]:
STAT_NAME = "z"
stat_map = meta_result.get_map(STAT_NAME)

In [ ]:
plot_stat_map(stat_map)

## Investigate what studies contributed to which clusters

In [ ]:
from nimare.diagnostics import FocusCounter # Only works for coordinate-based meta-analyses
fc = FocusCounter(target_image=STAT_NAME)

In [ ]:
contribution_table, cluster_img = fc.transform(meta_result)

In [ ]:
contribution_table